# Video Processing for BT Lab
Video files to png
<br>
<br>
![UofC logo](./pictures/uofc_logo-black.jpg)

In [55]:
#import libraries
import os
import re
import cv2
import json
import pandas as pd
import numpy as np

In [35]:
user_drive = input("Enter user drive: ")
video_path = f"{user_drive}:/Christian/DI_centre_structured"
repo_dir = os.getcwd()
json_dir = repo_dir + "/records/JSON"

In [36]:
def run_video_to_frame(video_path:str , save_folder:str , step: int):
    video = cv2.VideoCapture(video_path)
    assert video.isOpened()
    video.release()

In [37]:
def get_frames_path(local_path: str, level:str) -> str:
    fixed_path = local_path.replace("\\", "/")
    fixed_path_split = fixed_path.split("/")
    video_folder = "/".join(fixed_path_split[:-1])
    video_filename = fixed_path_split[-1].split(".")[0]
    folder_path = video_folder + f"/frames_{video_filename}_{level}"
    return folder_path

In [58]:
def down_sample(old_fps, new_fps):
    down_sampled_list = []
    user_frames = np.arange(0, old_fps, dtype=int)
    step = old_fps / new_fps

    for fps in range(new_fps):
        start = int(fps * step)
        end = int((fps + 1) * step)
        
        if fps == new_fps - 1:
            end = old_fps # ensures that the last element is included
        
        avg = sum(user_frames[start:end]) // (end - start)
        down_sampled_list.append(avg)

    return down_sampled_list

In [39]:
def up_sample(old_fps, new_fps):
    pass

In [60]:
def convert_video_to_frame(all_patients:dict, rgb_fps: dict):
    for patient_id, patient_info in all_patients.items():
        try:
            video_path = patient_info["local path"]
            old_fps = int(patient_info["old fps"])

            for level, new_fps in rgb_fps.items():
                frames_folder = get_frames_path(video_path, level)

                if old_fps <= new_fps:
                    frames_to_pick = up_sample(old_fps, new_fps)
                else:
                    frames_to_pick = down_sample(old_fps, new_fps)
                
                if len(frames_to_pick) != new_fps:
                    raise ValueError("Number of frames to pick is not equal to new fps")
        except Exception as e:
            print(f"type error: {e} for patient {video_path}")

        # run_video_to_frame(video_path, frames_folder, new_fps)
        break;

In [41]:
def load_json(json_dir:str, filename:str) -> dict:
    full_path = json_dir + "/" + filename

    with open(full_path, "r") as json_data:
        data = json.load(json_data)

    return(data)

## Video to frames (stage 3.1)

Based on the desired fps, turn videos into frames

In [42]:
""" local vals"""

rgb_fps = {
    "lower_bound": 10,
    "upper_bound": 20
}

thermal_fps = {
    "lower_bound": 5,
    "upper_bound": 10
}

In [48]:
""" load JSON files """

metadata_rgb = load_json(json_dir, "rgb_complete.json")
metadata_thermal = load_json(json_dir, "thermal_complete.json")

In [61]:
""" convert video to png (rgb) """
convert_video_to_frame(metadata_rgb, rgb_fps)